## Cabeçalho
### Objetivo
Apresentar alguns insights da fato_despesas

**Sobre**
**Data de crição:** 13/10/2025

**Responsável:** Wandrieli Nery Barbosa

In [0]:
%run /Users/wandrieli.89@gmail.com/projeto_tiller/Parametros/nb_parametros

In [0]:
df_despesas = spark.table("tiller.gold.fato_despesas") # Carrega a tabela fato_despesas

display(df_despesas.limit(5))

Quantidade de registros e estatísticas básicas

In [0]:
# Contagem total de registros
total_registros = df_despesas.count()
print(f"Total de registros: {total_registros:,}")

# Estatísticas descritivas das colunas numéricas
display(df_despesas.select("valor_documento", "valor_liquido", "valor_glosa").describe())


Gráfico: top 10 deputados por valor total de despesas

In [0]:
# Agrupar por deputado e somar as despesas
df_top_deputados = (
    df_despesas.groupBy("id_deputado")
    .agg(F.sum("valor_liquido").alias("total_liquido"))
    .orderBy(F.desc("total_liquido"))
    .limit(10)
    .toPandas()
)

# Plotar
plt.figure(figsize=(10, 5))
plt.barh(df_top_deputados["id_deputado"], df_top_deputados["total_liquido"], color="teal")
plt.title("Top 10 Deputados por Valor Total de Despesas")
plt.xlabel("Valor total líquido (R$)")
plt.ylabel("ID do Deputado")
plt.gca().invert_yaxis()
plt.show()


Gráfico: total de despesas por estado (UF)

In [0]:
df_por_estado = (
    df_despesas.groupBy("sigla_uf")
    .agg(F.sum("valor_liquido").alias("total_liquido"))
    .orderBy(F.desc("total_liquido"))
    .toPandas()
)

plt.figure(figsize=(10, 5))
plt.bar(df_por_estado["sigla_uf"], df_por_estado["total_liquido"], color="orange")
plt.title("Total de Despesas por Estado (UF)")
plt.xlabel("UF")
plt.ylabel("Total Líquido (R$)")
plt.xticks(rotation=45)
plt.show()


Gráfico: evolução mensal de despesas

In [0]:
df_mensal = (
    df_despesas.groupBy("ano", "mes")
    .agg(F.sum("valor_liquido").alias("total_liquido"))
    .orderBy("ano", "mes")
    .toPandas()
)

df_mensal["periodo"] = df_mensal["ano"].astype(str) + "-" + df_mensal["mes"].astype(str).str.zfill(2)

plt.figure(figsize=(10, 4))
plt.plot(df_mensal["periodo"], df_mensal["total_liquido"], marker="o")
plt.title("Evolução Mensal das Despesas")
plt.xlabel("Período (Ano-Mês)")
plt.ylabel("Total Líquido (R$)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


Verificação de qualidade dos dados

In [0]:
# Valores nulos por coluna
df_nulls = df_despesas.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df_despesas.columns])
display(df_nulls)

# Quantidade de CPFs/CNPJs inválidos
df_invalid_cnpj = df_despesas.filter(F.length("cnpj_cpf_fornecedor") < 11)
print(f"Registros com CNPJ/CPF inválido: {df_invalid_cnpj.count()}")

# Faixas de valor (para detectar outliers)
display(df_despesas.select("valor_liquido").summary())
